# Its an Analytics vidhya Competition 
### To get an idea about competition and use data set for your own Notebooks  
* Dataset Link- https://www.kaggle.com/vetrirah/customer
* Competition Link - https://datahack.analyticsvidhya.com/contest/janatahack-customer-segmentation/#ProblemStatement

I personally Achieced Rank 8 on Public leaderboard and 25 on Private leaderboard,But some of my other solutions can give rank 3 and 6 on private leader board which i missed to select,One more important think about this competition there is small amount of data so making a right validation set is also important( rank 2 on public leader board recieved rank 52 on private leader board in this competition)

In [ ]:
# import Libraries
import pandas as pd
import numpy as np
import matplotlib as pyplot
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# to see all the comands result in a single kernal 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# to increase no. of rows and column visibility in outputs
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [ ]:
# Model_01
# just found 1 date leak and get result as 92.47 with rank of 7 on public leader board
# i have seen that around 89% of date IDs are repeated to merge test and train data sort them with ids and 
# given next outcome value to current one and fill missing value with D
# Model_02
# make an model and 
# if the same id is not present in test data set then fill its value by predicted value 

In [ ]:
# Import data
train = pd.read_csv(r'../input/customer/Train.csv')
test = pd.read_csv(r'../input/customer/Test.csv')
sample_submmission = pd.read_csv(r'../input/customer/sample_submission.csv')

### Basic Visualization

In [ ]:
# Having a look at data
train.head()
test.head()
sample_submmission.head()
train.shape ,test.shape, sample_submmission.shape

In [ ]:
# to check type of variables and missing columns in dataset
train.info()
test.info()

 Apart from target column there are 6 other columns which are object type and there are also missing values in data

In [ ]:
# checking missing values 
(100*train.isna().sum().sort_values(ascending=False)/len(train)).round(2)
(100*test.isna().sum().sort_values(ascending=False)/len(test)).round(2)

6 columns has missing values and Work_Experience has around 10% values mising

In [ ]:
# Checking Distribution of Target variable
sns.countplot(train['Segmentation'])

It's a balanced distribution 

In [ ]:
train.columns

In [ ]:
train['ID'].nunique()
test['ID'].nunique()
df['ID'].nunique()

In [ ]:
#Merging train and test data
train['is_train']=1
test['is_train']=0
df=pd.concat([train,test],axis=0)
df.shape

#In testset and train set all ids are unique as their no. of rows equal to no. of unique values of Id 
#but some of Train set Id's get repeated in Test as no. of unique id in df not equal to sum of test train unique ids
#exact no.approx 89 %(2332 observations)
#assuming outcome remain same if ID. is same
#These below article may help in understanding this 
#Approach- https://www.kaggle.com/questions-and-answers/171939 
#Approach in Detail in this competition- https://www.kaggle.com/questions-and-answers/171953

# Model 1

In [ ]:
train.sort_values(by=['ID'],inplace=True)
test.sort_values(by=['ID'],inplace=True)
df.sort_values(by=['ID','is_train'],inplace=True)

In [ ]:
 df['Segmentation_output'] = df['Segmentation'].shift(-1)

In [ ]:
Encoding = df.groupby('ID').size()
df['ID_Freq_encoded']= df.ID.map(Encoding)

In [ ]:
out=df[df['is_train']==0]

In [ ]:
out['Segmentation_output']=out['Segmentation_output'].fillna('D')

In [ ]:
col=['ID','ID_Freq_encoded','Segmentation_output']
out_1=out[col]

# 2nd Method

In [ ]:
df['Ever_Married']=df['Ever_Married'].replace({'Yes':1,'No':0})
df['Gender']=df['Gender'].replace({'Male':1,'Female':0})
df['Graduated']=df['Graduated'].replace({'Yes':1,'No':0})
df['Spending_Score']=df['Spending_Score'].replace({'Low':0,'Average':1,'High':2})
df['Var_1']=df['Var_1'].str[-1:]

# Visulization

In [ ]:
train.head()
train.nunique()
train['Segmentation'].value_counts()

In [ ]:
train.describe().T
test.describe().T

In [ ]:
# Missing values
train.isna().sum()/len(train)
test.isna().sum()/len(test)

In [ ]:
train['Ever_Married']=train['Ever_Married'].replace({'Yes':1,'No':0})
train['Gender']=train['Gender'].replace({'Male':1,'Female':0})
train['Graduated']=train['Graduated'].replace({'Yes':1,'No':0})
train['Spending_Score']=train['Spending_Score'].replace({'Low':0,'Average':1,'High':2})

In [ ]:
Encoding = df.groupby('Profession').size()
df['Profession_Freq_encoded']= df.Profession.map(Encoding)
Encoding = df.groupby('Var_1').size()
df['Var_1_Freq_encoded']= df.Var_1.map(Encoding)

In [ ]:
df_var=pd.get_dummies(df['Var_1'])
df_Profession=pd.get_dummies(df['Profession'])
df_Segmentation=pd.get_dummies(df['Segmentation'])

In [ ]:
df_var.shape, df_Profession.shape , df.shape

In [ ]:
df_1=pd.concat([df,df_var,df_Profession,df_Segmentation],axis=1)

In [ ]:
df_1.head()
df_1.shape

In [ ]:
df_1=df_1.drop(['Profession','Var_1'],axis=1)

In [ ]:
train_1=df_1[df_1['is_train']==1]
test_1=df_1[df_1['is_train']==0]
test_id=test_1['ID']

In [ ]:
train_1=train_1.drop('ID',axis=1)

In [ ]:
train_1=train_1.drop(['is_train','Segmentation_output','ID_Freq_encoded','A','B','C','D'],axis=1)
test_1=test_1.drop(['Segmentation','is_train','Segmentation_output','ID_Freq_encoded','A','B','C','D'],axis=1)

In [ ]:
Y=train_1['Segmentation']
train_1=train_1.drop('Segmentation',axis=1)

# Model

In [ ]:
from sklearn.model_selection import train_test_split
X_t, X_tt, y_t, y_tt = train_test_split(train_1, Y, test_size=.25, random_state=2)

In [ ]:
from lightgbm import LGBMClassifier
lgbcl = LGBMClassifier(n_estimators=60)
lgbcl= lgbcl.fit(X_t, y_t,eval_metric='multi_error',eval_set=(X_tt , y_tt),verbose=10)
y_predict = lgbcl.predict(X_tt)
print(lgbcl.score(X_t , y_t))
print(lgbcl.score(X_tt , y_tt))

In [ ]:
feat_importances = pd.Series(lgbcl.feature_importances_, index=X_t.columns)
#feat_importances.nlargest(30).plot(kind='barh')
feat_importances.nsmallest(100).plot(kind='barh')
plt.show()

In [ ]:
test_ID=test_1['ID']
test_1=test_1.drop('ID',axis=1)
lgbcl= lgbcl.fit(train_1, Y)
y_predict = lgbcl.predict(test_1)

In [ ]:
out_2 = pd.DataFrame({ 'ID':test_ID,'Segmentation_1':y_predict}) 
#submission.to_csv("1st_try.csv", index = False)
out=pd.merge(out_1,out_2,how='inner', on='ID')

In [ ]:
out['Segmentation']=out['Segmentation_output'][out['ID_Freq_encoded']==2]
out['Segmentation']=out['Segmentation'].fillna(out['Segmentation_1'])

In [ ]:
col=['ID','Segmentation']
output=out[col]
output.to_csv("sol.csv", index = False)

# If you find thsi notebook helpful don't forget to upvote. It will Enourage me to make some more Notebooks like this.